### Test residual block 


In [1]:
from eeg_net.eeg_resnet import * 
from eeg_net.eeg_net_base import *
import numpy as np 
import matplotlib.pyplot as plt 
from torchsummary import summary
%load_ext autoreload
%autoreload 2
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 200

In [ ]:
conv= conv3x3(in_channels=32, out_channels=64)
print(conv)
dummy = torch.ones((1,1,1,1))
print(dummy)
block = ResidualBlock(1,64)
block(dummy)

In [ ]:
dummy = torch.ones((1,32,224,224))
block = ResNetBasicBlock(32,64)
print(block(dummy).shape)
print(block)

### ResNoteBottleNeckBlock test 

In [ ]:
dummy = torch.ones((1,32,10,10))
block = ResNetBottleNeckBlock(32,64)
block(dummy).shape
print(block)

### ResNetLayer test 

In [ ]:
dummy = torch.ones((1, 64, 48, 48))

layer = ResNetLayer(64, 128, block=ResNetBasicBlock, n=3)
layer(dummy).shape

In [2]:
X_test = np.load("data/X_test.npy")
y_test = np.load("data/y_test.npy")
person_train_valid = np.load("data/person_train_valid.npy")
X_train_valid = np.load("data/X_train_valid.npy")
y_train_valid = np.load("data/y_train_valid.npy")
person_test = np.load("data/person_test.npy")

print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))
X_test_dir = './data/X_test.npy'
y_test_dir = './data/y_test.npy' 
X_train_valid_dir = './data/X_train_valid.npy' 
y_train_valid_dir = './data/y_train_valid.npy'


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [ ]:
model = EEGResNet(1,4)
options = {
    'train_batch_size': 32,
    'scheduler_patience': 100,
    'weight_decay': 0.15,
    'val_batch_size': 1 
}
train_val_data = eeg_train_val_loader(
    X_train_valid_dir, y_train_valid_dir,device='cpu')
train_loader= train_val_data['train']
val_loader = train_val_data['val']
x ,y = [],[]
for data in train_loader:
    x,y = data 
    break 



In [ ]:
model = EEGResNet(1,4).to('cuda')
options = {
    'train_batch_size': 32,
    'scheduler_patience': 100,
    'weight_decay': 0.15,
    'val_batch_size': 1,
    'learning_rate': 1e-3,
    'epoch_num': 150,
}
train_val_data = eeg_train_val_loader(
    X_train_valid_dir, y_train_valid_dir)
loss_fn = nn.CrossEntropyLoss()
logs,_= train(model, options, loss_fn,
        data_dir=X_train_valid_dir,
        label_dir=y_train_valid_dir,
        preload_gpu=True)


### Data Preprocess 
#### shift_positive

In [ ]:
model = EEGResNet(1,4).to('cuda')
options = {
    'train_batch_size': 32,
    'scheduler_patience': 80,
    'weight_decay': 0.15,
    'val_batch_size': 1,
    'learning_rate': 1e-3,
    'epoch_num': 300,
}
train_val_data = eeg_train_val_loader(
    X_train_valid_dir, y_train_valid_dir)
loss_fn = nn.CrossEntropyLoss()
logs,_= train(model, options, loss_fn,
        data_dir=X_train_valid_dir,
        label_dir=y_train_valid_dir,
        preload_gpu=True)

In [ ]:
model = EEGResNetV2(1,4).to('cuda')
options = {
    'train_batch_size': 32,
    'scheduler_patience': 80,
    'weight_decay': 0.15,
    'val_batch_size': 1,
    'learning_rate': 1e-4,
    'epoch_num': 350,
    'transform': 'none'
}
train_val_data = eeg_train_val_loader(
    X_train_valid_dir, y_train_valid_dir)
loss_fn = nn.CrossEntropyLoss()
logs,_= train(model, options, loss_fn,
        data_dir=X_train_valid_dir,
        label_dir=y_train_valid_dir,
        preload_gpu=True)

In [2]:
model = ShallowConv(1,4)
summary(model.cuda(),(1,22,1000))
model = EEGResNet(1,4)
summary(model.cuda(),(1,22,1000))

NameError: name 'ShallowConv' is not defined

In [3]:
Encoder = ResNetEncoder(in_channels=1,block_sizes=[32,64],deepths=[1,1])
summary(Encoder.cuda(),(1,22,1000))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 32, 11, 500]           1,568
       BatchNorm2d-2          [-1, 32, 11, 500]              64
              ReLU-3          [-1, 32, 11, 500]               0
         MaxPool2d-4           [-1, 32, 6, 250]               0
        Conv2dAuto-5           [-1, 32, 6, 250]           9,216
       BatchNorm2d-6           [-1, 32, 6, 250]              64
              ReLU-7           [-1, 32, 6, 250]               0
        Conv2dAuto-8           [-1, 32, 6, 250]           9,216
       BatchNorm2d-9           [-1, 32, 6, 250]              64
             ReLU-10           [-1, 32, 6, 250]               0
 ResNetBasicBlock-11           [-1, 32, 6, 250]               0
      ResNetLayer-12           [-1, 32, 6, 250]               0
           Conv2d-13           [-1, 64, 6, 125]           2,048
      BatchNorm2d-14           [-1, 64,

In [4]:
model = EEGResNetV2(in_channels=1,classes=4,block_sizes=[16,32,64],deepths=[1,1,1])

summary(model.encoder.cuda(),(1,22,1000))
print(model.encoder.blocks[-1].blocks[-1].expanded_channels)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 16, 22, 976]             400
       BatchNorm2d-2          [-1, 16, 22, 976]              32
               ELU-3          [-1, 16, 22, 976]               0
         MaxPool2d-4          [-1, 16, 22, 243]               0
        Conv2dAuto-5          [-1, 16, 22, 243]           2,304
       BatchNorm2d-6          [-1, 16, 22, 243]              32
               ELU-7          [-1, 16, 22, 243]               0
        Conv2dAuto-8          [-1, 16, 22, 243]           2,304
       BatchNorm2d-9          [-1, 16, 22, 243]              32
              ELU-10          [-1, 16, 22, 243]               0
EEGResNetBasicBlock-11          [-1, 16, 22, 243]               0
      ResNetLayer-12          [-1, 16, 22, 243]               0
           Conv2d-13          [-1, 32, 22, 122]             512
      BatchNorm2d-14          [-1, 32

In [5]:
model = EEGResNetV2(in_channels=1,classes=4,block_sizes=[32,64,32,18],deepths=[1,1,1,1])
summary(model.cuda(),(1,22,1000))


RuntimeError: mat1 dim 1 must match mat2 dim 0

In [6]:
model = EEGResNetV2(in_channels=1,classes=4,block_sizes=[40,32,18,18],deepths=[1,1,1,1]).to('cuda')

options = {
    'train_batch_size': 32,
    'scheduler_patience': 80,
    'weight_decay': 0.15,
    'val_batch_size': 1,
    'learning_rate': 1e-4,
    'epoch_num': 350,
    #'transform': 'shift_posistive',
}
train_val_data = eeg_train_val_loader(
    X_train_valid_dir, y_train_valid_dir)
loss_fn = nn.CrossEntropyLoss()
logs,_= train(model, options, loss_fn,
        data_dir=X_train_valid_dir,
        label_dir=y_train_valid_dir,
        preload_gpu=True)

Start training...
Epoch	Train Loss	Train Acc	Test Loss	Test_Acc	


RuntimeError: mat1 dim 1 must match mat2 dim 0